In [ ]:
import numpy as np
import cv2
import torch
import torch.utils.data
import os
import pickle
import json

In [49]:
tsfm_path = '/home/xymeng/Repo/jax3d/jax3d/projects/mobilenerf/get_rays/rays_inverse_false.json'
with open(tsfm_path, 'r') as jf:
    data_inv_f = json.load(jf)

tsfm_path_t = '/home/xymeng/Repo/jax3d/jax3d/projects/mobilenerf/get_rays/rays_inverse_true.json'
with open(tsfm_path_t, 'r') as jft:
    data_inv_t = json.load(jft)

FileNotFoundError: [Errno 2] No such file or directory: '/home/xymeng/Repo/jax3d/jax3d/projects/mobilenerf/get_rays/rays_inverse_false.json'

In [3]:
data_inv_f = torch.tensor(data_inv_f)

In [4]:
data_inv_f.shape

torch.Size([512, 512, 3, 1])

In [5]:
data_inv_t = torch.tensor(data_inv_t)
data_inv_t.shape

torch.Size([512, 512, 3, 1])

In [6]:
torch.unique(data_inv_f[...,0,:]==data_inv_t[...,0,:])

tensor([True])

In [12]:
torch.unique(data_inv_f[...,1:,:]+data_inv_t[...,1:,:])

tensor([0., 1.])

In [14]:
data_inv_f[...,1:,:].shape, data_inv_t[...,1:,:].shape

(torch.Size([512, 512, 2, 1]), torch.Size([512, 512, 2, 1]))

In [24]:
abs(data_inv_f[...,1:,:]).mean(), # data_inv_t[...,1:,:]
# (data_inv_f[...,2:3,:]+data_inv_t[...,2:3,:]).sum()

(tensor(128.5000),)

In [82]:
focal = 527
width=height=512

k_zju = np.array([[537.14068604,   0.        , 271.41711426],
       [  0.        , 537.71148682, 242.44180298],
       [  0.        ,   0.        ,   1.        ]])

k = np.array([
      [1./focal, 0, -.5 * width / focal],
      [0, -1./focal, .5 * height / focal],
      [0, 0, -1.],
  ])
kneg = np.array([
      [1./focal, 0, .5 * width / focal],
      [0, 1./focal, -.5 * height / focal],
      [0, 0, 1.],
  ])

neg = np.array([
      [1., 0, 0],
      [0, -1., 0],
      [0, 0, -1.],
  ])

np.linalg.inv(k_zju), k, kneg, np.matmul(kneg, neg), np.matmul(np.linalg.inv(k_zju), neg), np.matmul(k_zju, np.linalg.inv(k_zju))

(array([[ 0.00186171,  0.        , -0.50529986],
        [ 0.        ,  0.00185973, -0.45087711],
        [ 0.        ,  0.        ,  1.        ]]),
 array([[ 0.00189753,  0.        , -0.4857685 ],
        [ 0.        , -0.00189753,  0.4857685 ],
        [ 0.        ,  0.        , -1.        ]]),
 array([[ 0.00189753,  0.        ,  0.4857685 ],
        [ 0.        ,  0.00189753, -0.4857685 ],
        [ 0.        ,  0.        ,  1.        ]]),
 array([[ 0.00189753,  0.        , -0.4857685 ],
        [ 0.        , -0.00189753,  0.4857685 ],
        [ 0.        ,  0.        , -1.        ]]),
 array([[ 0.00186171,  0.        ,  0.50529986],
        [ 0.        , -0.00185973,  0.45087711],
        [ 0.        ,  0.        , -1.        ]]),
 array([[1.00000000e+00, 0.00000000e+00, 5.68434189e-14],
        [0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]))

In [38]:
## actual inv_K for ZJU
DeviceArray([[ 0.00186171,  0.        ,  0.50529987],
             [ 0.        , -0.00185973,  0.4508771 ],
             [ 0.        ,  0.        , -1.        ]], dtype=float32)


In [84]:
-np.ones_like([3,3,2])

array([-1, -1, -1])

In [61]:
inverse_y_mul = np.array([
      [1., 0, 0],
      [0, -1., 0],
      [0, 0, -1.],
    ])
K_zju = np.asarray([[537.1407,   0.0000, 271.4171],
            [  0.0000, 537.7115, 242.4418],
            [  0.0000,   0.0000,   1.0000]])
inv_K = np.linalg.inv(K_zju)
# st()
np.matmul(inv_K, inverse_y_mul)

array([[ 0.00186171,  0.        ,  0.50529982],
       [ 0.        , -0.00185973,  0.4508771 ],
       [ 0.        ,  0.        , -1.        ]])

In [43]:
torch.unique(kt==kf)

tensor([True])

In [74]:
xyz = torch.rand(10,10,3,1)
xyz1 = xyz.clone()
xyz1[...,1:,:] = xyz1[...,1:,:]*(-1)
xyz2 = torch.matmul(torch.tensor(neg).float(), xyz.float())
torch.tensor(neg), torch.unique(xyz2==xyz1)

(tensor([[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]], dtype=torch.float64),
 tensor([True]))

In [80]:
xyz3 = torch.matmul(torch.matmul(torch.tensor(k_zju).float(), torch.tensor(neg).float()), xyz.float())
xyz4 = torch.matmul(torch.tensor(k_zju).float(), xyz2)
torch.unique(xyz3==xyz4)

tensor([True])

In [2]:
import numpy as np
import jax.numpy as jnp

# Define a matrix A
A = jnp.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

# Compute the Moore-Penrose pseudo-inverse of A using NumPy
A_pinv_np = np.linalg.pinv(A)

# Convert the NumPy array to a JAX array
A_pinv = jnp.array(A_pinv_np)

# Multiply A by its pseudo-inverse (should give an identity-like matrix)
result = jnp.matmul(A, A_pinv)

print("A:")
print(A)
print("\nPseudo-inverse of A:")
print(A_pinv)
print("\nA * A_pinv:")
print(result)


A:
[[1. 2.]
 [3. 4.]
 [5. 6.]]

Pseudo-inverse of A:
[[-1.3333334  -0.33333334  0.6666667 ]
 [ 1.0833334   0.33333334 -0.4166667 ]]

A * A_pinv:
[[ 0.8333334   0.33333334 -0.16666669]
 [ 0.3333335   0.33333337  0.33333325]
 [-0.16666698  0.33333325  0.8333335 ]]


In [1]:
import numpy as np
import jax.numpy as jnp
# Load data from the saved file
param_fname = '/data/xymeng/Repo/humannerf/sample_motion_arrays.npz'
with np.load(param_fname) as data:
    jax_arrays = {key: jnp.array(value) for key, value in data.items()}
# Create variables with the same name as the dictionary keys and assign the JAX arrays
for key, value in jax_arrays.items():
    exec(f"{key} = value")

In [2]:
rays = (jnp.array([512, 512, 4, 3]), jnp.array([512, 512, 4, 3]))

In [8]:
def _sample_motion_fields_forward_warp(
        pts,
        motion_scale_Rs,
        motion_Ts,
        motion_weights_vol,
        cnl_bbox_min_xyz, cnl_bbox_scale_xyz,
        output_list):
    orig_shape = list(pts.shape)
    pts = pts.reshape(-1, 3)  # [N_rays x N_samples, 3]

    # remove BG channel
    motion_weights = motion_weights_vol[:-1]

    weights_list = []
    for i in range(motion_weights.shape[0]):  # for each motion in total 24 motions

        # forward_warp:
        pos = pts
        pos = (pos - cnl_bbox_min_xyz[None, :]) \
            * cnl_bbox_scale_xyz[None, :] - 1.0

        # Use JAX's map_coordinates to replace torch's grid_sample
        weights = map_coordinates(motion_weights[jnp.newaxis, i:i + 1, :, :, :], 
                                  pos[jnp.newaxis, jnp.newaxis, jnp.newaxis, :, :], 
                                  order=1)
        weights = weights[0, 0, 0, 0, :, jnp.newaxis]  # (4194304, 1)
        weights_list.append(weights)

    backwarp_motion_weights = jnp.concatenate(weights_list, axis=-1)  # for each point, there is a weight for each motion
    total_bases = backwarp_motion_weights.shape[-1]

    backwarp_motion_weights_sum = jnp.sum(backwarp_motion_weights,
                                          dim=-1, keepdims=True)
    weighted_motion_fields = []
    for i in range(total_bases):
        # forward warp:
        # using jax linalg: pos = linalg.matmul(linalg.inv(motion_scale_Rs[i, :, :]), (pts - motion_Ts[i, :]).T).T 
        ## FIXME: modified using numpuy, potential risk in gradient flow
        pos = np.linalg.matmul(np.linalg.inv(motion_scale_Rs[i, :, :]), (pts - motion_Ts[i, :]).T).T
        pos = jnp.array(pos)
        weighted_pos = backwarp_motion_weights[:, i:i + 1] * pos
        weighted_motion_fields.append(weighted_pos)
    x_skel = jnp.sum(
        jnp.stack(weighted_motion_fields, dim=0), dim=0
    ) / backwarp_motion_weights_sum.clip(min=0.0001)
    # the final pos is the weighted avg of all motions
    fg_likelihood_mask = backwarp_motion_weights_sum

    x_skel = x_skel.reshape(orig_shape[:2] + [3])
    backwarp_motion_weights = \
        backwarp_motion_weights.reshape(orig_shape[:2] + [total_bases])
    fg_likelihood_mask = fg_likelihood_mask.reshape(orig_shape[:2] + [1])

    results = {}

    if 'x_skel' in output_list:  # [N_rays x N_samples, 3]
        results['x_skel'] = x_skel
    if 'fg_likelihood_mask' in output_list:  # [N_rays x N_samples, 1]
        results['fg_likelihood_mask'] = fg_likelihood_mask

    return results

def sample_motion_fields_forward_warp_mobile_nerf(
            rays,
            motion_scale_Rs,
            motion_Ts,
            motion_weights_vol,
            cnl_bbox_min_xyz,
            cnl_bbox_scale_xyz,
):
  ## first reverse yz
  rays = [ jnp.stack([_ray[..., 0], _ray[..., 2], _ray[..., 1]], axis=-1) for _ray in rays]
  
  ## begin warping
  # reshape into (N, 3)
  rays = [ x.reshape(1, -1, 3) for x in rays]
  
  # warping using weight vol
  sample_motion_outs = []
  for pts in rays:
    mv_output = _sample_motion_fields_forward_warp(
                      pts=pts,
                      motion_scale_Rs=motion_scale_Rs[0], 
                      motion_Ts=motion_Ts[0], 
                      motion_weights_vol=motion_weights_vol,
                      cnl_bbox_min_xyz=cnl_bbox_min_xyz, 
                      cnl_bbox_scale_xyz=cnl_bbox_scale_xyz,
                      output_list=['x_skel', 'fg_likelihood_mask'])
    pts_mask = mv_output['fg_likelihood_mask']
    cnl_pts = mv_output['x_skel'].reshape(-1,3)
    sample_motion_outs.append(cnl_pts)
  
  rays = sample_motion_outs

  ## recover shape
  rays = [ x.reshape(512,512, 4,3) for x in rays]
  ## last reverse yz
  rays = [ jnp.stack([_ray[..., 0], _ray[..., 2], _ray[..., 1]], axis=-1) for _ray in rays]




In [12]:
from jax.scipy.ndimage import map_coordinates
warped_rays = sample_motion_fields_forward_warp_mobile_nerf(
    rays=rays,
    motion_scale_Rs=motion_scale_Rs,
    motion_Ts=motion_Ts,
    motion_weights_vol=motion_weights_vol,
    cnl_bbox_min_xyz=cnl_bbox_min_xyz,
    cnl_bbox_scale_xyz=cnl_bbox_scale_xyz,
  )


ValueError: coordinates must be a sequence of length input.ndim, but 1 != 5

In [30]:
import jax
import jax.numpy as jnp
import jax.scipy as jscipy
import numpy as np
import torch
import torch.nn.functional as F

# Toy input image
image = jnp.arange(16).reshape((4, 4)).astype(jnp.float32)

# Define the sampling coordinates
coords = jnp.array([[0.5, 0.5], [1.5, 1.5], [2.5, 2.5]]).reshape((3, 2))

# JAX: Using map_coordinates
interpolated_jax = jscipy.ndimage.map_coordinates(image, coords.T, order=1)

# Convert JAX array to NumPy array
image_np = jnp.asarray(image)
image_np = np.array(image_np)

# Convert coordinates to NumPy array and then PyTorch tensor
coords_np = np.array(coords)
grid = (coords_np / (np.array(image.shape[::-1]) - 1) * 2 - 1).reshape((1, 3, 2))
grid_torch = torch.from_numpy(grid).float()

# PyTorch: Using grid_sample
image_torch = torch.from_numpy(image_np)
# interpolated_torch = F.grid_sample(image_torch.unsqueeze(0).unsqueeze(0), grid_torch, align_corners=True).squeeze()

print("Interpolated Values (JAX):", interpolated_jax)
# print("Interpolated Values (PyTorch):", interpolated_torch)


Interpolated Values (JAX): [ 2.5  7.5 12.5]


In [13]:
import jax
import jax.numpy as jnp
import jax.scipy as jscipy

# Toy input image
image = jnp.arange(32*32*32).reshape((1, 1, 32, 32, 32)).astype(jnp.float32)

# Define correct number of dimensions in the sampling coordinates
coords3 = jnp.array([[0.5, 0.5], [0.5, 0.5]])[jnp.newaxis, jnp.newaxis, :, :].reshape(1,1,1,1,1)  # Three-dimensional coordinates
print(coords3.shape, len(coords3))
# JAX: Using map_coordinates
interpolated_jax = jscipy.ndimage.map_coordinates(image, coords3.T, order=1)


(1, 1, 4, 1, 1) 1


ValueError: coordinates must be a sequence of length input.ndim, but 1 != 5

: 